IMPORTS

In [65]:
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
from datetime import date
import plotly.express as px
import plotly.graph_objects as go
import time
import datetime
import numpy as np
import pandas as pd
from datetime import date
import base64
from jupyter_dash import JupyterDash
import networkx as nx
import matplotlib.pyplot as plt
vertex_list = pd.read_excel('List of Vertices.xlsx')
import plotly.figure_factory as ff

APP UTILS

In [104]:
def Header():
    return html.Div([get_header(), get_menu()])

def get_header():
    header = html.Div(
        [
            html.Div([
                dbc.Row(
                    
                    [
                        dbc.Col(html.Img(
                            src="bird_logo.jpg",
                            className="logo",
                            height="130",
                            style={"float": "left"}
                        ), width=2),
                        dbc.Col([
                            html.Br([]),
                            html.Br([]),
                            html.H1(" COMP 152VA Lekagul Bird Park Dashboard",
                                    style = {"text-align": "center"})
                        ], width={"size":2, "offset":3}),
                    ])
            ])
        ])
    return header

def get_menu():
    menu = html.Div([
        dbc.NavbarSimple(
            children=[
                dbc.NavItem(dbc.NavLink("Exploratory Data Analysis", href="http://127.0.0.1:7090/")),
                dbc.NavItem(dbc.NavLink("Speeding", href="http://127.0.0.1:7090/speeding")),
                dbc.NavItem(dbc.NavLink("Camping", href="http://127.0.0.1:7090/camping")),
                dbc.NavItem(dbc.NavLink("Rangers' Negligence", href="http://127.0.0.1:7090/rangers")),
                dbc.NavItem(dbc.NavLink("Night Visits", href="http://127.0.0.1:7090/night_visits")),
                dbc.NavItem(dbc.NavLink("Other Questionable Activity", href="http://127.0.0.1:7090/q-activity"))
            ],
            color='lightgrey',
            className="our_navbar",
            #style = {"margin-right": "200px"},
            #fluid = True
    )
])
    return menu


def create_graph_card(heading, graph_id, style_list):
    card = dbc.Card(
            dbc.CardBody([
                html.H3(str(heading), style={"text-align": "center"}),
                html.Div(dcc.Graph(id = graph_id),)
            ]),
            style = {"height": style_list[0],  
                     "margin-left": style_list[1], 
                     "margin-right": style_list[2],
                     "margin-top": style_list[3],
                    }
            )
    return card

BASIC VARIABLE CREATION

In [77]:
# Adding additional variables as deemed necessary
df_speeding = pd.read_excel("Lekagul Sensor Data Pairs by Car-ID (Speed and Distance).xlsx")
df_full_path = pd.read_excel("Lekagul Sensor Data by Car-ID.xlsx")
                             
df_speeding["day"] = df_speeding["Timestamp"].dt.day
df_speeding["month"] = df_speeding["Timestamp"].dt.month
df_speeding["year"] = df_speeding["Timestamp"].dt.year
df_speeding["path"] = df_speeding["gate-name"] + "_" + df_speeding["next_gate"]
df_speeding["month_year"] = df_speeding["Timestamp"].dt.strftime('%m/%Y')
df_speeding["hour"] = df_speeding["Timestamp"].dt.hour
df_speeding["week_day"] = df_speeding["Timestamp"].dt.weekday

# Getting min and max dates of the dataset
min_date = str(df_speeding['Timestamp'].min()).split(" ")[0]
max_date = str(df_speeding['Timestamp'].max()).split(" ")[0]

min_day, min_month, min_year = int(min_date.split("-")[2]),int(min_date.split("-")[1]),int(min_date.split("-")[0])
max_day, max_month, max_year = int(max_date.split("-")[2]),int(max_date.split("-")[1]),int(max_date.split("-")[0])

all_car_ids = df_full_path['car-id'].unique().tolist()
frame = df_full_path[['car-id','car-path']]
illegal_exit_id_list = []
for i in range(len(all_car_ids)):
    curr_frame = df_full_path[df_full_path['car-id'] == all_car_ids[i]]
    frame_path = curr_frame['car-path'].iloc[0].split("-")
    last_item = frame_path[-1]
    if(last_item.__contains__("e") == False):
        illegal_exit_id_list.append(all_car_ids[i])

NETWORKX SCRIPTING

In [194]:
def DrawGraph(my_dict):
    node_x = []
    node_y = []
    edge_x = []
    edge_y = []
    pos_dict = {}
    G = nx.DiGraph()
    # create a list of all vertices that we want to add from my_dict
    list_of_keys = list(my_dict.keys())
    # set for all nodes we want to add
    all_nodes_to_add = set()
    for i in range(len(list_of_keys)):
        curr_path = my_dict[list_of_keys[i]]
        prev_node = None
        for j in range(len(curr_path)):
            all_nodes_to_add.add(curr_path[j])
            curr_x = vertex_list[vertex_list['Abbrev'] == curr_path[j]]['X-coordinate on Map'].iloc[0]
            curr_y = vertex_list[vertex_list['Abbrev'] == curr_path[j]]['Y-coordinate on Map'].iloc[0]*-1
            pos_dict[curr_path[j]] = (curr_x,curr_y)
            G.add_node(curr_path[j], X = curr_x, Y = curr_y)
            if(j > 0):
                prev_node = curr_path[j-1]
                G.add_edge(prev_node,curr_path[j])


    for node in G.nodes():
        node_x.append(G.nodes[node]['X'])
        node_y.append(G.nodes[node]['Y'])

    for edge in G.edges():
        if(edge[0] != edge[1]):
            edge_x.append(int(G.nodes[edge[0]]['X']))
            edge_y.append(int(G.nodes[edge[0]]['Y']))
            edge_x.append(int(G.nodes[edge[1]]['X']))
            edge_y.append(int(G.nodes[edge[1]]['Y']))  

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines')

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        hoverinfo='text',
        marker=dict(
            showscale=False,
            colorscale='Earth',
            reversescale=True,
            color=[],
            size=10,
            line_width=2))
    node_text = []

    node_list = list(G.nodes)
    for node in node_list:
        node_text.append(node)

    node_trace.text = node_text

    fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    showarrow=True,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                ).update_traces(textposition='bottom center').update_layout(width = 500)

    return fig

EDA LAYOUT AND GRAPH CREATION -- APP IS CALLED FROM HERE

In [195]:
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.SUPERHERO])
app.config.suppress_callback_exceptions = True

# =============================================================================
# EDA LAYOUT CREATION SECTION
# =============================================================================


def create_eda_layout():
    return html.Div([
        Header(),
        dbc.Row([
            dbc.Col(
                # This is the Card for the control
                dbc.Card([
                    # Control Beginning
                    html.H6("Select Car type"),
                    dcc.Dropdown(
                        id="type_dropdown",  # These IDs are important for the callbacks
                        options=[{'label': i, 'value': i} for i in df_speeding["car-type"].unique()],
                        style={"padding-left": "24px"},
                        multi=True,
                        value=df_speeding["car-type"].unique()  # By default all values are selected
                    ),
                    html.Br([]),
                    html.H6("Define timeframe"),
                    dcc.DatePickerRange(
                        id='date_picker',
                        min_date_allowed=date(min_year, min_month, min_day),
                        max_date_allowed=date(max_year, max_month, max_day),
                        initial_visible_month=date(min_year, min_month, min_day),
                        start_date=date(min_year, min_month, min_day),
                        end_date=date(max_year, max_month, max_day),
                        style={"margin-left": "24px"}
                    ),
                 ], style={"height": "600px",
                          "margin-left": "24px",
                          "margin-right": "12px",
                          "margin-top": "24px",
                          "padding": "24px"})
                , width={"size": 3}),
            dbc.Col([
                # This is the card for the pie chart
                dbc.Card(
                    dbc.CardBody([
                        html.H3("Activity-by-Month", style={"text-align": "center"}),
                        dcc.Graph(
                            id="graph1",
                        )
                    ]), style={
                        "height": "600px",
                        "margin-left": "12px",
                        "margin-right": "24px",
                        "margin-top": "24px",
                    }
                )],
                width=4),
            dbc.Col([
                # This is the card for the pie chart
                dbc.Card(
                    dbc.CardBody([
                        html.H3("Activity-by-Hour", style={"text-align": "center"}),
                        dcc.Graph(
                            id="graph2",
                        )
                    ]), style={
                        "height": "600px",
                        "margin-left": "12px",
                        "margin-right": "24px",
                        "margin-top": "24px",
                    }
                )],
                width=5)
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    # This is the card for the first bar chart
                    dbc.CardBody([
                        html.H3("Activity-by-Weekday", style={"text-align": "center"}),
                        html.Br([]),
                        dcc.Graph(
                            id="graph3",
                        )
                    ]), style={"margin-left": "24px",
                               "margin-right": "12px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
            dbc.Col([
                dbc.Card(
                    # This is the card for the second pie chart
                    dbc.CardBody([
                        html.H3("Activity-by-Gate", style={"text-align": "center"}),
                        html.Br([]),
                        html.Div(
                            dcc.Graph(
                                id="graph4",
                            ),
                        )
                    ]), style={"margin-left": "12px",
                               "margin-right": "24px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
        ]),
    ])



# =============================================================================
# EDA GRAPH CREATION CREATION SECTION
# =============================================================================

@app.callback(Output('graph1', 'figure'),
              [Input("type_dropdown", "value"),
               Input('date_picker', 'start_date'),
               Input('date_picker', 'end_date'),
               ]
)

# Activity by Month graph
def activity_by_month(types, start, end):
    filtered_df = df_speeding[df_speeding["car-type"].isin(types)]
    filtered_df = filtered_df[filtered_df["Timestamp"] > start]
    filtered_df = filtered_df[filtered_df["next_timestamp"] <= end] 
    grouped_df = filtered_df[["month_year", "car-type", "Timestamp"]]
    grouped_df = grouped_df.groupby(["month_year","car-type"]).count().reset_index()
    grouped_df.columns = ['Month/Year','Car-Type', 'Activity']
    grouped_df['Month/Year'] = pd.to_datetime(grouped_df['Month/Year'], format='%m/%Y')
    grouped_df = grouped_df.sort_values(by = 'Month/Year', ascending = True)
    fig = px.histogram(grouped_df, x= "Month/Year", y = 'Activity', color = 'Car-Type')
    fig = fig.update_xaxes(rangeslider_visible=True)
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    fig.update_xaxes(nticks=13)
    return fig

@app.callback(Output('graph2', 'figure'),
              [Input("type_dropdown", "value"),
               Input('date_picker', 'start_date'),
               Input('date_picker', 'end_date'),
               ]
)
# activity by hour
def activity_by_hour(types, start, end):
    filtered_df = df_speeding[df_speeding["car-type"].isin(types)]
    filtered_df = filtered_df[filtered_df["Timestamp"] > start]
    filtered_df = filtered_df[filtered_df["next_timestamp"] <= end] 
    filtered_df['Timestamp'] = pd.to_datetime(filtered_df['Timestamp'] )
    filtered_df['Timestamp'] = filtered_df['Timestamp'].dt.strftime('%m/%d/%Y')
    grouped_df = filtered_df[["month_year", "hour", "Timestamp"]]
    grouped_df = grouped_df.groupby(["month_year", "hour"]).count().reset_index()
    grouped_df.columns = ['Month/Year', 'Hour', 'Activity']
    grouped_df['Month/Year'] = pd.to_datetime(grouped_df['Month/Year'])
    grouped_df = grouped_df.sort_values(by = 'Month/Year', ascending = True)
    grouped_df['Month/Year'] = grouped_df['Month/Year'].dt.strftime('%m/%Y')
    fig = px.area(grouped_df, x="Hour", y = "Activity", color = "Month/Year")
    fig = fig.update_xaxes(rangeslider_visible=True)
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    fig.update_xaxes(nticks=24)
    return fig

@app.callback(Output('graph3', 'figure'),
              [Input("type_dropdown", "value"),
               Input('date_picker', 'start_date'),
               Input('date_picker', 'end_date'),
               ]
)

# activity by gate
def activity_by_weekday(types, start, end):
    filtered_df = df_speeding[df_speeding["car-type"].isin(types)]
    filtered_df = filtered_df[filtered_df["Timestamp"] > start]
    filtered_df = filtered_df[filtered_df["next_timestamp"] <= end] 
    filtered_df['Timestamp'] = pd.to_datetime(filtered_df['Timestamp'] )
    filtered_df['Timestamp'] = filtered_df['Timestamp'].dt.strftime('%m/%d/%Y')
    grouped_df = filtered_df[["month_year", "week_day", "Timestamp"]]
    grouped_df = grouped_df.groupby(["month_year", "week_day"]).count().reset_index()
    grouped_df.columns = ['Month/Year', 'Weekday', 'Activity']
    grouped_df['Month/Year'] = pd.to_datetime(grouped_df['Month/Year'])
    grouped_df = grouped_df.sort_values(by = 'Month/Year', ascending = True)
    grouped_df['Month/Year'] = grouped_df['Month/Year'].dt.strftime('%m/%Y')
    fig = px.bar(grouped_df, x='Weekday', y = 'Activity', color = "Month/Year")
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

@app.callback(Output('graph4', 'figure'),
              [Input("type_dropdown", "value"),
               Input('date_picker', 'start_date'),
               Input('date_picker', 'end_date'),
               ]
)
def activity_by_gate(types, start, end):
    filtered_df = df_speeding[df_speeding["car-type"].isin(types)]
    filtered_df = filtered_df[filtered_df["Timestamp"] > start]
    filtered_df = filtered_df[filtered_df["next_timestamp"] <= end] 
    grouped_df = filtered_df[["gate-name", "car-type", "hour", "Timestamp"]]
    grouped_df = grouped_df.groupby(["gate-name", "car-type", "hour"]).count().reset_index()
    grouped_df.columns = ['Gate Name', 'Car-Type', 'Hour', 'Count']
    grouped_df = grouped_df.sort_values(by = "Hour", ascending = True)
    fig = px.bar(grouped_df, x= "Gate Name", y = "Count", color = 'Car-Type', animation_frame = "Hour")
    fig["layout"].pop("updatemenus")
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig








SPEEDING LAYOUT AND GRAPH CREATION SECTION

In [196]:
def create_speeding_layout():
    return html.Div([
        Header(),
        dbc.Row([
            dbc.Col(
                # This is the Card for the control
                dbc.Card([
                    # Control Beginning
                    html.H6("Select Car type"),
                    dcc.Dropdown(
                        id="type_dropdown1",  # These IDs are important for the callbacks
                        options=[{'label': i, 'value': i} for i in df_speeding["car-type"].unique()],
                        style={"padding-left": "24px"},
                        multi=True,
                        value=df_speeding["car-type"].unique()  # By default all values are selected
                    ),
                    html.Br([]),
                    html.H6("Define timeframe"),
                    dcc.DatePickerRange(
                        id='date_picker1',
                        min_date_allowed=date(min_year, min_month, min_day),
                        max_date_allowed=date(max_year, max_month, max_day),
                        initial_visible_month=date(min_year, min_month, min_day),
                        start_date=date(min_year, min_month, min_day),
                        end_date=date(max_year, max_month, max_day),
                        style={"margin-left": "24px"}
                    ),
                 ], style={"height": "600px",
                          "margin-left": "24px",
                          "margin-right": "12px",
                          "margin-top": "24px",
                          "padding": "24px"})
                , width={"size": 2}),
            dbc.Col([
                # This is the card for the pie chart
                dbc.Card(
                    dbc.CardBody([
                        html.H3("Top 10 Edges Where Speeding Occurs", style={"text-align": "center"}),
                        dcc.Graph(
                            id="graph5",
                        )
                    ]), style={
                        "height": "600px",
                        "margin-left": "12px",
                        "margin-right": "24px",
                        "margin-top": "24px",
                    }
                )],
                width=4),
            dbc.Col([
                # This is the card for the pie chart
                dbc.Card(
                    dbc.CardBody([
                        html.H3("Top 10 Speeding Offenders", style={"text-align": "center"}),
                        dcc.Graph(
                            id="graph6",
                        )
                    ]), style={
                        "height": "600px",
                        "margin-left": "12px",
                        "margin-right": "24px",
                        "margin-top": "24px",
                    }
                )],
                width=6)
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    # This is the card for the first bar chart
                    dbc.CardBody([
                        html.H3("Speeding Instances by Car-Type", style={"text-align": "center"}),
                        html.Br([]),
                        dcc.Graph(
                            id="graph7",
                        )
                    ]), style={"margin-left": "24px",
                               "margin-right": "12px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
            dbc.Col([
                dbc.Card(
                    # This is the card for the second pie chart
                    dbc.CardBody([
                        html.H3("Speeding Prevalence by Edge", style={"text-align": "center"}),
                        html.Br([]),
                        html.Div(
                            dcc.Graph(
                                id="graph8",
                            ), style={'overflowY':'scroll', 'height':500}
                        )
                    ]), style={"margin-left": "12px",
                               "margin-right": "24px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    # This is the card for the first bar chart
                    dbc.CardBody([
                        html.H3("Racer Activity", style={"text-align": "center"}),
                        html.Br([]),
                        dcc.Graph(
                            figure = drag_racers(),
                        )
                    ]), 
                )],
                width=6),
            dbc.Col([
                dbc.Card(
                    # This is the card for the second pie chart
                    dbc.CardBody([
                        html.H3("Racing Path", style={"text-align": "center"}),
                        html.Br([]),
                        html.Div(
                            dcc.Graph(
                                figure = drag_racers_graph(),
                            ), style={'width': 500, 'height':500}
                        )
                    ]), style={"margin-left": "12px",
                               "margin-right": "24px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
        ]),
        
    ])

# =============================================================================
# UPDATE SPEEDING GRAPHS SECTION
# =============================================================================


@app.callback(Output('graph5', 'figure'),
              [Input("type_dropdown1", "value"),
               Input('date_picker1', 'start_date'),
               Input('date_picker1', 'end_date'),
               ]
)

# Top 10 Paths where speeding occurs histogram
def top10_speeding_paths(types, start, end):
    filtered_df = df_speeding[df_speeding["car-type"].isin(types)]
    filtered_df = filtered_df[filtered_df["Timestamp"] > start]
    filtered_df = filtered_df[filtered_df["next_timestamp"] <= end] 

    filtered_df = filtered_df[filtered_df["speed_bool"] ==1]

    grouped_df = filtered_df[["car-type", "path", "speed_bool"]]
    top10_df = grouped_df.groupby(["path"]).sum()

    top10_df = top10_df.nlargest(10, 'speed_bool')
    top10_paths = top10_df.index.tolist()

    new_top10_df = grouped_df[grouped_df["path"].isin(top10_paths)]
    grouped_df = new_top10_df.groupby(["path", "car-type"]).sum()
    grouped_df = grouped_df.reset_index()
    grouped_df = grouped_df.sort_values(by = 'speed_bool', ascending = False)
    grouped_df.columns = ['Edge Travelled', 'Car-Type', 'Total Speeding Instances']

    fig = px.bar(grouped_df, x= "Edge Travelled", y = "Total Speeding Instances", color="Car-Type")
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

@app.callback(Output('graph6', 'figure'),
              [Input("type_dropdown1", "value"),
               Input('date_picker1', 'start_date'),
               Input('date_picker1', 'end_date'),
               ]
)
# Top 10 speeding offenders -- DONE -- SPEEDING
def top10_speeding_offenders(types, start, end):

    filtered_df = df_speeding[df_speeding["car-type"].isin(types)]
    filtered_df = filtered_df[filtered_df["Timestamp"] > start]
    filtered_df = filtered_df[filtered_df["next_timestamp"] <= end] 

    filtered_df = filtered_df[df_speeding["speed_bool"] ==1]
    grouped_df = filtered_df[["car-id", "car-type", "path", "speed_bool"]]
    speeding_offenders = grouped_df.groupby(["car-id"]).sum().reset_index()
    speeding_offenders = speeding_offenders.sort_values(by = ['speed_bool'], ascending = False).head(10)
    top_10_offenders_list = speeding_offenders['car-id'].values.tolist()

    # get top speeds
    max_speed_list = []
    start_date = []
    end_date = []
    for i in range(len(top_10_offenders_list)):
        curr_id = top_10_offenders_list[i]
        max_speed_list.append(df_speeding[df_speeding['car-id'] == curr_id]['avg_speed'].max())
        start_date.append(str(df_speeding[df_speeding['car-id'] == curr_id]['Timestamp'].min()))
        end_date.append(str(df_speeding[df_speeding['car-id'] == curr_id]['next_timestamp'].max()))

    speeding_offenders['max_speed_clocked'] = max_speed_list
    speeding_offenders['timestamp_first_seen'] = start_date
    speeding_offenders['timestamp_last_seen'] = end_date

    speeding_offenders = speeding_offenders[['car-id','timestamp_first_seen',
           'timestamp_last_seen','speed_bool', 'max_speed_clocked']]
    speeding_offenders.columns = ['Car-ID', 'First Seen at Park', 'Last Seen at Park', '# Speeding Violations', 'Top Speed Clocked']
    speeding_offenders['Top Speed Clocked'] = np.round(speeding_offenders['Top Speed Clocked'], decimals = 2)
    speeding_offenders['# Speeding Violations'] = np.round(speeding_offenders['# Speeding Violations'], decimals = 0)

    colorscale = [[0, '#4d004c'],[.5, '#f2e5ff'],[1, '#ffffff']]

    fig =  ff.create_table(speeding_offenders, colorscale=colorscale)
    fig.update_layout(autosize = True)
    
    return fig

@app.callback(Output('graph7', 'figure'),
              [Input("type_dropdown1", "value"),
               Input('date_picker1', 'start_date'),
               Input('date_picker1', 'end_date'),
               ]
)
# Top 10 Speeding Offenders datatable
# Speeding instances by car-type -- DONE -- decided on filter -- SPEEDING
def speeding_instances_car_type(types,  start, end):
    filtered_df = df_speeding[df_speeding["car-type"].isin(types)]
    filtered_df = filtered_df[filtered_df["Timestamp"] > start]
    filtered_df = filtered_df[filtered_df["next_timestamp"] <= end] 

    filtered_df = filtered_df[filtered_df["speed_bool"] ==1]
    grouped_df = filtered_df[["car-type", "speed_bool"]]
    speeding_instances_by_car_type = grouped_df.groupby(["car-type"]).sum()
    speeding_instances_by_car_type = speeding_instances_by_car_type.reset_index()
    speeding_instances_by_car_type.columns = ['Car-Type', 'No of Speeding Instances']
    fig = px.bar(speeding_instances_by_car_type, x="Car-Type", y = "No of Speeding Instances", color = 'Car-Type')
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig


@app.callback(Output('graph8', 'figure'),
              [Input("type_dropdown1", "value"),
               Input('date_picker1', 'start_date'),
               Input('date_picker1', 'end_date'),
               ]
)

# Total Speeding Instances in TimeFrame
def speeding_prevalence(types, start, end):
    filtered_df = df_speeding[df_speeding["car-type"].isin(types)]
    filtered_df = filtered_df[filtered_df["Timestamp"] > start]
    filtered_df = filtered_df[filtered_df["next_timestamp"] <= end] 
                                  
    filtered_df = filtered_df[["path", "speed_bool"]]
    path_list = filtered_df["path"].unique().tolist()
    speeding_count_at_path = []
    total_count_path_travelled = []
    for i in range(len(path_list)):
        curr_path = path_list[i]
        curr_df = filtered_df[filtered_df["path"] ==  curr_path]
        speed_curr_df = curr_df[curr_df["speed_bool"] == 1]
        total_count_path_travelled.append(len(curr_df))
        speeding_count_at_path.append(len(speed_curr_df))

    speeding_prevalence = pd.DataFrame(path_list)
    speeding_prevalence['Total Speeding Instances'] = speeding_count_at_path
    speeding_prevalence['Total Times Travelled'] = total_count_path_travelled
    cols = speeding_prevalence.columns.tolist()
    cols[0] = "Edge"
    speeding_prevalence.columns = cols
    speeding_prevalence['Speeding Prevalence Ratio'] = speeding_prevalence['Total Speeding Instances']/speeding_prevalence['Total Times Travelled']
    speeding_prevalence = speeding_prevalence.sort_values(by = 'Speeding Prevalence Ratio', ascending = False).head(20)
    fig = px.bar(speeding_prevalence, y= "Edge", x = "Speeding Prevalence Ratio", orientation = 'h', color = "Edge")
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

def drag_racers():
    filtered_df = df_speeding
    racer_car_ids = filtered_df[filtered_df['avg_speed'] > 40]['car-id'].values.tolist()
    path_of_racers = filtered_df[filtered_df['car-id'].isin(racer_car_ids)]
    path_of_racers = path_of_racers[['car-id','car-type','gate-name','Timestamp','next_gate','next_timestamp','avg_speed']]
    path_of_racers.columns = ['Car-ID', 'Car-Type', 'Gate 1', 'Timestamp at Gate 1', 'Gate 2', 'Timestamp at Gate 2', 'Speed Between Gates']
    path_of_racers['path'] = path_of_racers['Gate 1'] + "-" + path_of_racers['Gate 2']

    fig = px.line(path_of_racers, x = "path", y = 'Speed Between Gates', color = 'Car-ID', hover_data = ['Timestamp at Gate 2'])
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

def drag_racers_graph():
    filtered_df = df_speeding
    racer_car_ids = filtered_df[filtered_df['avg_speed'] > 40]['car-id'].values.tolist()
    df_full_path = pd.read_excel("Lekagul Sensor Data by Car-ID.xlsx")
    racer_dict = {}
    racer_dict[racer_car_ids[0]] = df_full_path[df_full_path['car-id'] == racer_car_ids[0]]['car-path'].iloc[0].split("-")
    racer_dict[racer_car_ids[1]] = df_full_path[df_full_path['car-id'] == racer_car_ids[1]]['car-path'].iloc[0].split("-")
    fig = DrawGraph(racer_dict)
    fig.update_layout(width = 850, height = 500)
    
    return fig



CAMPING POPULARITY

In [197]:
# layout creation
def create_camping_layout():
    return html.Div([
        Header(),
        dbc.Row([
                dbc.Col(
                # This is the Card for the control
                    dbc.Card([
                        # Control Beginning
                        html.H6("Select Car type"),
                        dcc.Dropdown(
                            id="type_dropdown2",  # These IDs are important for the callbacks
                            options=[{'label': i, 'value': i} for i in df_speeding[df_speeding["car-type"] != "2P"]['car-type'].unique()],
                            style={"padding-left": "24px"},
                            multi=True,
                            value=df_speeding["car-type"].unique()  # By default all values are selected
                        ),
                        html.Br([]),
                        html.H6("Define timeframe"),
                        dcc.DatePickerRange(
                            id='date_picker2',
                            min_date_allowed=date(min_year, min_month, min_day),
                            max_date_allowed=date(max_year, max_month, max_day),
                            initial_visible_month=date(min_year, min_month, min_day),
                            start_date=date(min_year, min_month, min_day),
                            end_date=date(max_year, max_month, max_day),
                            style={"margin-left": "24px"}
                        ),
                     ], style={"height": "400px",
                              "margin-left": "24px",
                              "margin-right": "12px",
                              "margin-top": "24px",
                              "padding": "24px"})
                    , width={"size": 2}),
                dbc.Col([
                # This is the card for the pie chart
                    dbc.Card(
                        dbc.CardBody([
                            html.H3("Camping Popularity Heatmap", style={"text-align": "center"}),
                            dcc.Graph(
                                id = 'graph_camping_hm',
                            )
                        ]), style={
                            "height": "900px",
                            "margin-left": "12px",
                            "margin-right": "24px",
                            "margin-top": "24px",
                        }
                    )],
                    width=6),
                dbc.Col([
                    # This is the card for the pie chart
                    dbc.Card(
                        dbc.CardBody([
                            html.H3("Camping Popularity Pie Chart", style={"text-align": "center"}),
                            dcc.Graph(
                                id = 'graph_camping_pie',
                            )
                        ]), style={
                            "height": "600px",
                            "margin-left": "12px",
                            "margin-right": "24px",
                            "margin-top": "24px",
                        }
                    )],
                    width=4)
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    # This is the card for the first bar chart
                    dbc.CardBody([
                        html.H3("Time Spent (Hours) At Camping Locations", style={"text-align": "center"}),
                        html.Br([]),
                        dcc.Graph(
                            id = 'graph_camping_box1',
                        )
                    ]), style={
                        "height": "600px","margin-left": "12px",
                               "margin-right": "12px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
            dbc.Col([
                dbc.Card(
                    # This is the card for the second pie chart
                    dbc.CardBody([
                        html.H3("Camping Popularity and Hours Spent For Single Night Campers", style={"text-align": "center"}),
                        html.Br([]),
                        html.Div(
                            dcc.Graph(
                                id = 'graph_camping_box2',
                            ), style={'overflowY':'scroll'}
                        )
                    ]), style={"height": "600px","margin-left": "12px",
                               "margin-right": "24px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
        ]),
    ])


# =============================================================================
# UPDATE CAMPING GRAPHS SECTION
# =============================================================================

@app.callback(Output('graph_camping_pie', 'figure'),
              [Input("type_dropdown2", "value"),
               Input('date_picker2', 'start_date'),
               Input('date_picker2', 'end_date'),
               ]
)
# camping popularity pie chart
def camping_pie(types,  start, end):
    
    filtered_df = df_speeding[df_speeding["car-type"].isin(types)]
    filtered_df = filtered_df[filtered_df["Timestamp"] > start]
    filtered_df = filtered_df[filtered_df["next_timestamp"] <= end] 
    filtered_df = df_speeding[df_speeding['car-type']!='2P']
    all_gates = df_speeding['gate-name'].unique().tolist()
    camping_spots = [s for s in all_gates if "camping" in s]
    filtered_df = filtered_df[filtered_df['gate-name'].isin(camping_spots)]
    filtered_df = filtered_df[['month_year','gate-name','car-type','Timestamp']]
    grouped_df = filtered_df.groupby(['month_year','gate-name','car-type']).count().reset_index()
    fig = px.pie(grouped_df, values = 'Timestamp', names = 'gate-name', hover_data = ['month_year'])
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

@app.callback(Output('graph_camping_hm', 'figure'),
              [Input("type_dropdown2", "value"),
               Input('date_picker2', 'start_date'),
               Input('date_picker2', 'end_date'),
               ]
)
# camping popularity potential heat map
def camping_popularity_heatmap(types, start, end):
    filtered_df = df_speeding[df_speeding["car-type"].isin(types)]
    filtered_df = filtered_df[filtered_df["Timestamp"] > start]
    filtered_df = filtered_df[filtered_df["next_timestamp"] <= end] 
    filtered_df = df_speeding[df_speeding['car-type']!='2P']
    all_gates = df_speeding['gate-name'].unique().tolist()
    camping_spots = [s for s in all_gates if "camping" in s]
    filtered_df = filtered_df[filtered_df['gate-name'].isin(camping_spots)]
    filtered_df = filtered_df[['Timestamp', 'gate-name','car-id']]
    filtered_df['Timestamp'] = filtered_df['Timestamp'].dt.strftime('%m/%d/%Y')
    filtered_df = filtered_df.sort_values(by = 'Timestamp', ascending = False)
    grouped_df = filtered_df.groupby(['Timestamp','gate-name']).count().reset_index()
    grouped_df = grouped_df.sort_values(by = 'Timestamp', ascending = True)
    grouped_df.columns = ['Date', 'Camping Spot', 'Activity Count']
    grouped_df['Date'] = pd.to_datetime(grouped_df['Date'])
    grouped_df.sort_values(by=['Date','Camping Spot'])

    fig = go.Figure(data=go.Heatmap(
            z=grouped_df['Activity Count'],
            x=grouped_df['Date'],
            y=grouped_df['Camping Spot'],
            colorscale='Viridis'))

    fig.update_layout(xaxis_nticks = 20, yaxis_nticks = 40, height = 800)
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

@app.callback(Output('graph_camping_box1', 'figure'),
              [Input("type_dropdown2", "value"),
               Input('date_picker2', 'start_date'),
               Input('date_picker2', 'end_date'),
               ]
)
def camping_time_spent(types,  start, end):
    filtered_df = df_speeding[df_speeding["car-type"].isin(types)]
    filtered_df = filtered_df[filtered_df["Timestamp"] > start]
    filtered_df = filtered_df[filtered_df["next_timestamp"] <= end] 
    all_gates = df_speeding['gate-name'].unique().tolist()
    camping_spots = [s for s in all_gates if "camping" in s]
    filtered_df = df_speeding
    filtered_df = df_speeding[df_speeding['car-type']!='2P']
    filtered_df = filtered_df[filtered_df['gate-name'] == filtered_df['next_gate']]
    filtered_df = filtered_df[filtered_df['gate-name'].isin(camping_spots)]
    filtered_df = filtered_df[['car-id','Timestamp','next_timestamp','month_year','gate-name','time_diff_hours']]
    filtered_df.columns = ['car-id','Timestamp','next_timestamp','month_year','Camping Spot','Time Spent at Camping Spot']
    # grouped_df = filtered_df.groupby(['car-id','Timestamp','next_timestamp','month_year','gate-name']).mean()
    fig = px.box(filtered_df, x = "Camping Spot", y = "Time Spent at Camping Spot")
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

@app.callback(Output('graph_camping_box2', 'figure'),
              [Input("type_dropdown2", "value"),
               Input('date_picker2', 'start_date'),
               Input('date_picker2', 'end_date'),
               ]
)

def camping_popularity_one_night(types,  start, end):
    filtered_df = df_speeding[df_speeding["car-type"].isin(types)]
    filtered_df = filtered_df[filtered_df["Timestamp"] > start]
    filtered_df = filtered_df[filtered_df["next_timestamp"] <= end] 
    all_gates = df_speeding['gate-name'].unique().tolist()
    camping_spots = [s for s in all_gates if "camping" in s]
    filtered_df = df_speeding
    filtered_df = df_speeding[df_speeding['car-type']!='2P']
    filtered_df = filtered_df[filtered_df['gate-name'] == filtered_df['next_gate']]
    filtered_df = filtered_df[filtered_df['gate-name'].isin(camping_spots)]
    filtered_df = filtered_df[['car-id','Timestamp','next_timestamp','month_year','gate-name','time_diff_hours']]
    filtered_df.columns = ['car-id','Timestamp','next_timestamp','month_year','Camping Spot','Time Spent at Camping Spot']
    filtered_df = filtered_df[filtered_df['Time Spent at Camping Spot'] < 24]

    fig = px.box(filtered_df, x = "Camping Spot", y = "Time Spent at Camping Spot")
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    fig.update_layout(autosize=True)
    return fig


RANGER MOVEMENTS

In [198]:
# layout creation
def create_ranger_layout():
    return html.Div([
        Header(),
        dbc.Row([
            dbc.Col(
                # This is the Card for the control
                dbc.Card([
                    html.H6("Select Car ID"),
                    dcc.Dropdown(
                        id="id_dropdown5",  # These IDs are important for the callbacks
                        options=[{'label': i, 'value': i} for i in df_speeding[df_speeding["car-type"] =="2P"]["car-id"].unique()],
                        style={"padding-left": "24px"},
                        multi=True,
                        value=None
                    ),
                    html.H6("Define timeframe"),
                    dcc.DatePickerRange(
                        id='date_picker5',
                        min_date_allowed=date(min_year, min_month, min_day),
                        max_date_allowed=date(max_year, max_month, max_day),
                        initial_visible_month=date(min_year, min_month, min_day),
                        start_date=date(min_year, min_month, min_day),
                        end_date=date(max_year, max_month, max_day),
                        style={"margin-left": "24px"}
                    ),
                 ], style={"height": "300px",
                          "margin-left": "24px",
                          "margin-right": "12px",
                          "margin-top": "24px",
                          "padding": "24px"})
                , width={"size": 3}),
            dbc.Col([
                # This is the card for the pie chart
                dbc.Card(
                    dbc.CardBody([
                        html.H3("Ranger Visit Heatmap", style={"text-align": "center"}),
                        dcc.Graph(
                            id = 'graph_ranger_heatmap',
                        )
                    ]), style={
                        "height": "800px",
                        "margin-left": "12px",
                        "margin-right": "24px",
                        "margin-top": "24px",
                    }
                )],
                width=9)
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    # This is the card for the first bar chart
                    dbc.CardBody([
                        html.H3("Ranger Visit Bar Chart Subplot", style={"text-align": "center"}),
                        html.Br([]),
                        dcc.Graph(
                            id = 'graph_ranger_subplot',
                        )
                    ]), style={"margin-left": "24px",
                               "margin-right": "12px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
            dbc.Col([
                dbc.Card(
                    # This is the card for the second pie chart
                    dbc.CardBody([
                        html.H3("Top 10 Ranger Excursions by Duration", style={"text-align": "center"}),
                        html.Br([]),
                        html.Div(
                            dcc.Graph(
                               id='graph_ranger_excursion',
                            ),
                        )
                    ]), style={"margin-left": "12px",
                               "margin-right": "24px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    # This is the card for the first bar chart
                    dbc.CardBody([
                        html.H3("Most Visited Edges by Rangers", style={"text-align": "center"}),
                        html.Br([]),
                        dcc.Graph(
                            id = 'graph_ranger_mostvisited',
                        )
                    ]), style={"margin-left": "24px",
                               "margin-right": "12px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
            dbc.Col([
                dbc.Card(
                    # This is the card for the second pie chart
                    dbc.CardBody([
                        html.H3("Least Visited Edges by Rangers", style={"text-align": "center"}),
                        html.Br([]),
                        html.Div(
                            dcc.Graph(
                               id='graph_ranger_leastvisited',
                            ),
                        )
                    ]), style={"margin-left": "12px",
                               "margin-right": "24px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
        ]),
    ])

# =============================================================================
# UPDATE RANGER GRAPHS SECTION
# =============================================================================

@app.callback(Output('graph_ranger_excursion', 'figure'),
              [Input("id_dropdown5", "value"),
               Input('date_picker5', 'start_date'),
               Input('date_picker5', 'end_date'),
               ]
)
def longest_ranger_excursions(ids,start,end):
    filtered_df = df_full_path
    filtered_df = filtered_df[filtered_df["car-type"] == "2P"]
    filtered_df = filtered_df[filtered_df["start-date"] >= start]
    filtered_df = filtered_df[filtered_df["end-date"] <= end] 
    if(ids):
        filtered_df = filtered_df[filtered_df["car-id"].isin(ids)] 
    filtered_df = filtered_df[['car-id', 'start-date', 'end-date','start-pt','end-pt']]
    filtered_df.columns = ['Car-ID', 'Start Date', 'End Date', 'Starting Point', "Ending Point"]
    filtered_df['Start Date'] = pd.to_datetime(filtered_df['Start Date'])
    filtered_df['End Date'] = pd.to_datetime(filtered_df['End Date'])
    filtered_df = filtered_df.sort_values(by = 'Start Date')
    filtered_df['Duration (Hours)'] = np.round((filtered_df['End Date'] - filtered_df['Start Date'])/ pd.Timedelta(hours=1),2)
    filtered_df = filtered_df.sort_values(by = 'Duration (Hours)', ascending = False)
    
    filtered_df['Start Date'] = filtered_df['Start Date'].dt.strftime('%m/%d/%Y %H:%M')
    filtered_df['End Date'] = filtered_df['End Date'].dt.strftime('%m/%d/%Y %H:%M')
    filtered_df = filtered_df.head(10) 
    colorscale = [[0, '#4d004c'],[.5, '#f2e5ff'],[1, '#ffffff']]
    fig =  ff.create_table(filtered_df, colorscale=colorscale)
    fig.update_layout(autosize = True)

    return fig

@app.callback(Output('graph_ranger_heatmap', 'figure'),
              [Input("id_dropdown5", "value"),
               Input('date_picker5', 'start_date'),
               Input('date_picker5', 'end_date'),
               ]
)

# ranger visit heatmap
def ranger_visit_heatmap(ids,start,end):
    filtered_df = df_speeding[df_speeding['car-type']=='2P']
    filtered_df = filtered_df[filtered_df["Timestamp"] >= start]
    filtered_df = filtered_df[filtered_df["Timestamp"] <= end] 
    if(ids):
        filtered_df = filtered_df[filtered_df["car-id"].isin(ids)] 
    filtered_df = filtered_df[['Timestamp', 'gate-name','car-id']]
    filtered_df['Timestamp'] = filtered_df['Timestamp'].dt.strftime('%m/%d/%Y')
    filtered_df = filtered_df.sort_values(by = 'Timestamp', ascending = False)
    grouped_df = filtered_df.groupby(['Timestamp','gate-name']).count().reset_index()
    grouped_df = grouped_df.sort_values(by = 'Timestamp', ascending = True)
    grouped_df.columns = ['Date', 'Gate Name', 'Visit Count']
    grouped_df['Date'] = pd.to_datetime(grouped_df['Date'])
    grouped_df = grouped_df.sort_values(by=['Date','Gate Name'])

    fig = go.Figure(data=go.Heatmap(
            z=grouped_df['Visit Count'],
            x=grouped_df['Date'],
            y=grouped_df['Gate Name'],
            colorscale='Inferno'))

    fig.update_layout(xaxis_nticks = 20, yaxis_nticks = 40, height = 600)
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

@app.callback(Output('graph_ranger_subplot', 'figure'),
              [Input("id_dropdown5", "value"),
               Input('date_picker5', 'start_date'),
               Input('date_picker5', 'end_date'),
               ]
)

def ranger_visit_bar_chart_subplot(ids, start,end):
    from plotly.subplots import make_subplots
    fig = make_subplots(rows=1, cols=3)
    all_gates = df_speeding['gate-name'].unique().tolist()
    camping_spots = [s for s in all_gates if "camping" in s]
    gate_spots = [s for s in all_gates if "gate" in s]
    ranger_spots = [s for s in all_gates if "ranger" in s]
    
    filtered_df = df_full_path[df_full_path["start-date"] >= start]
    filtered_df = filtered_df[filtered_df["end-date"] <= end] 
    filtered_df = df_speeding[df_speeding['car-type']=='2P']
    if(ids):
        filtered_df = filtered_df[filtered_df["car-id"].isin(ids)] 
    filtered_df = filtered_df[['gate-name','car-id']]

    filtered_df_camping = filtered_df[filtered_df['gate-name'].isin(camping_spots)]
    filtered_df_gate = filtered_df[filtered_df['gate-name'].isin(gate_spots)]
    filtered_df_ranger = filtered_df[filtered_df['gate-name'].isin(ranger_spots)]

    grouped_df_camping = filtered_df_camping.groupby(['gate-name']).count().reset_index()
    grouped_df_camping.columns = ['Gate Name', 'Ranger Visit Frequency']
    grouped_df_camping = grouped_df_camping.sort_values(by = 'Ranger Visit Frequency', ascending = True)

    grouped_df_gate = filtered_df_gate.groupby(['gate-name']).count().reset_index()
    grouped_df_gate.columns = ['Gate Name', 'Ranger Visit Frequency']
    grouped_df_gate = grouped_df_gate.sort_values(by = 'Ranger Visit Frequency', ascending = True)

    grouped_df_ranger = filtered_df_ranger.groupby(['gate-name']).count().reset_index()
    grouped_df_ranger.columns = ['Gate Name', 'Ranger Visit Frequency']
    grouped_df_ranger = grouped_df_ranger.sort_values(by = 'Ranger Visit Frequency', ascending = True)


    fig.add_trace(go.Bar(y = grouped_df_camping['Ranger Visit Frequency'], x = grouped_df_camping['Gate Name']),row=1,col=1)
    fig.add_trace(go.Bar(y = grouped_df_gate['Ranger Visit Frequency'], x = grouped_df_gate['Gate Name']),row=1,col=2)
    fig.add_trace(go.Bar(y = grouped_df_ranger['Ranger Visit Frequency'], x = grouped_df_ranger['Gate Name']),row=1,col=3)      

    fig.update_layout(showlegend=False)
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig


@app.callback(Output('graph_ranger_mostvisited', 'figure'),
              [
               Input('date_picker5', 'start_date'),
               Input('date_picker5', 'end_date'),
               ]
)

def top10_most_visited_edges(start,end):
    filtered_df = df_speeding
    filtered_df = filtered_df[filtered_df["car-type"] == '2P']
    filtered_df = filtered_df[filtered_df["Timestamp"] >= start]
    filtered_df = filtered_df[filtered_df["Timestamp"] <= end] 
    filtered_df = filtered_df[["car-id", "path"]]
    grouped_df = filtered_df.groupby(["path"]).count().reset_index()
    grouped_df = grouped_df[~grouped_df["path"].str.contains('ranger')]
    grouped_df.columns = ['Edge Travelled', 'Count']
    bottom_10_df = grouped_df.nsmallest(10, 'Count')
    top10_df = grouped_df.nlargest(10, 'Count')

    fig = px.bar(top10_df, x= "Edge Travelled", y = "Count", color="Count")
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

@app.callback(Output('graph_ranger_leastvisited', 'figure'),
              [
               Input('date_picker5', 'start_date'),
               Input('date_picker5', 'end_date'),
               ]
)
def top10_least_visited_edges(start,end):
    filtered_df = df_speeding
    filtered_df = filtered_df[filtered_df["car-type"] == '2P']
    filtered_df = filtered_df[filtered_df["Timestamp"] >= start]
    filtered_df = filtered_df[filtered_df["Timestamp"] <= end] 
    grouped_df = filtered_df[["car-id", "path"]]
    grouped_df = grouped_df.groupby(["path"]).count().reset_index()
    grouped_df = grouped_df[~grouped_df["path"].str.contains('ranger')]
    grouped_df.columns = ['Edge Travelled', 'Count']
    bottom_10_df = grouped_df.nsmallest(10, 'Count')
    top10_df = grouped_df.nlargest(10, 'Count')

    fig = px.bar(bottom_10_df, x= "Edge Travelled", y = "Count", color="Count")
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig


NIGHT VISITS

In [199]:
def create_n_visits_layout():
    return html.Div([
        Header(),
        dbc.Row([
                dbc.Col([
                # This is the card for the pie chart
                    dbc.Card(
                        dbc.CardBody([
                            html.H3("Night Visitors with < 1 hour Stay", style={"text-align": "center"}),
                            dcc.Graph(
                                figure = night_walkers(),
                            )
                        ]), style={
                            "height": "800px",
                            "margin-left": "12px",
                            "margin-right": "24px",
                            "margin-top": "24px",
                        }
                    )],
                    width=4),
                dbc.Col([
                    # This is the card for the pie chart
                    dbc.Card(
                        dbc.CardBody([
                            html.H3("Four Axle Truck Night Visits < 1 hour", style={"text-align": "center"}),
                            dcc.Graph(
                                figure = four_truck_night_walker(),
                            )
                        ]), style={
                            "height": "800px",
                            "margin-left": "12px",
                            "margin-right": "24px",
                            "margin-top": "24px",
                        }
                    )],
                    width=8)
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    # This is the card for the first bar chart
                    dbc.CardBody([
                        html.H3("Illegal Gate Visit by Four Axle Truck", style={"text-align": "center"}),
                        html.Br([]),
                        dcc.Graph(
                            figure = four_truck_gate_visit(),
                        )
                    ]), style={"margin-left": "12px",
                               "margin-right": "12px",
                               "margin-top": "24px",
                               }
                )],
                width=4),
            dbc.Col([
                dbc.Card(
                    # This is the card for the second pie chart
                    dbc.CardBody([
                        html.H3("Possible Culprit Activity", style={"text-align": "center"},),
                        html.Br([]),
                        html.Div(
                            dcc.Graph(
                                figure = four_axle_culp(),
                            ),
                        )
                    ]), style={"margin-left": "12px",
                               "margin-right": "24px",
                               "margin-top": "24px",
                               }
                )],
                width=8),
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    # This is the card for the first bar chart
                    dbc.CardBody([
                        html.H3("Graph Path of Possible Culprit (7/23/2015)", style={"text-align": "center"}),
                        html.Br([]),
                        dcc.Graph(
                            figure = four_axle_culp_graph(),
                        )
                    ]), style={"margin-left": "12px",
                               "margin-right": "12px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
            dbc.Col([
                dbc.Card(
                    # This is the card for the first bar chart
                    dbc.CardBody([
                        html.H3("Graph Path of Possible Culprit (12/24/2015)", style={"text-align": "center"}),
                        html.Br([]),
                        dcc.Graph(
                            figure = four_axle_culp_graph_2(),
                        )
                    ]), style={"margin-left": "12px",
                               "margin-right": "12px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
        ]),
    ])

# =============================================================================
# UPDATE NIGHT VISIT GRAPHS SECTION
# =============================================================================


def night_walkers():
    filtered_df = df_full_path[df_full_path['duration-stay-hours'] < 1]
    filtered_df = filtered_df[(filtered_df['start-ts'].dt.hour >= 0) & (filtered_df['start-ts'].dt.hour < 6)]
    filtered_df = filtered_df[['car-type','duration-stay']]
    grouped_df_count = filtered_df.groupby(by = 'car-type').count().reset_index()
    grouped_df_count.columns = ['Car-Type', 'No of Distinct Visits']

    fig = px.bar(grouped_df_count, y='No of Distinct Visits' , x = 'Car-Type', color = 'Car-Type')
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

def four_truck_night_walker():
    filtered_df = df_full_path
    filtered_df = filtered_df[(filtered_df['start-ts'].dt.hour >= 0) & (filtered_df['start-ts'].dt.hour < 6)]
    filtered_df = filtered_df[filtered_df['car-type'] == '4']
    filtered_df = filtered_df.sort_values(by = ['start-date'])
    filtered_df = filtered_df[filtered_df['duration-stay-hours'] <1]
    fig = go.Figure(data=go.Heatmap(
                z=filtered_df['duration-stay-hours'],
                x=filtered_df['start-date'],
                y=filtered_df['car-id'],
                colorscale='Viridis'))

    fig.update_layout(xaxis_nticks = 20, yaxis_nticks = 40, height = 600, showlegend = True)
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

def four_truck_gate_visit():
    filtered_df = df_speeding
    filtered_df = filtered_df[filtered_df['car-type'] == '4']
    cant_visit_gates = ['gate0', 'gate1', 'gate2', 'gate3', 'gate4', 'gate5', 'gate6', 'gate7','gate8']
    filtered_df = filtered_df[filtered_df['gate-name'].isin(cant_visit_gates)]
    filtered_df = filtered_df[['car-id','hour', 'gate-name']]
    grouped_df = filtered_df.groupby(['gate-name','hour']).count().reset_index()
    grouped_df = grouped_df.sort_values(by = "hour", ascending = True)
    grouped_df.columns = ['Gate Name', 'Hour', 'Count']
    fig = px.bar(grouped_df, x= "Gate Name", y = "Count",  animation_frame = "Hour", color = "Gate Name")
    fig["layout"].pop("updatemenus")
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

def four_axle_culp():
    from plotly.subplots import make_subplots
    fig = make_subplots(rows=1, cols=3)

    filtered_df = df_full_path[df_full_path['car-path'] == 'e3-g6-rs6-g5-gg5-g3-rs3-rs3-g3-gg5-g5-rs6-g6-e3']
    filtered_df["month_year"] = pd.to_datetime(filtered_df["start-ts"].dt.strftime('%m/%Y'))
    filtered_df['arrival_hour'] = filtered_df["start-ts"].dt.hour
    filtered_df['arrival_weekday'] = filtered_df["start-ts"].dt.weekday
    filtered_df = filtered_df[['car-id','month_year','arrival_hour','arrival_weekday']]
    filtered_df.columns = ['Car-ID', 'Month/Year', 'Arrival Hour', 'Arrival Weekday']

    night_culp_by_month = filtered_df[['Car-ID', 'Month/Year']].groupby('Month/Year').count().reset_index()
    night_culp_by_month.columns = ['Month/Year','Visit Count']
    night_culp_by_hour = filtered_df[['Car-ID', 'Arrival Hour']].groupby('Arrival Hour').count().reset_index()
    night_culp_by_hour.columns = ['Arrival Hour','Visit Count']
    night_culp_by_weekday = filtered_df[['Car-ID', 'Arrival Weekday']].groupby('Arrival Weekday').count().reset_index()
    night_culp_by_weekday.columns = ['Arrival Weekday','Visit Count']

    fig.add_trace(go.Bar(y = night_culp_by_month['Visit Count'], x = night_culp_by_month['Month/Year']),row=1,col=1)
    fig.add_trace(go.Bar(y = night_culp_by_hour['Visit Count'], x = night_culp_by_hour['Arrival Hour']),row=1,col=2)
    fig.add_trace(go.Bar(y = night_culp_by_weekday['Visit Count'], x = night_culp_by_weekday['Arrival Weekday']),row=1,col=3)      

    fig.update_layout(showlegend=False)
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig


def four_axle_culp_graph():
    filtered_df = df_full_path
    culp_ids = filtered_df[filtered_df['car-path'] == 'e3-g6-rs6-g5-gg5-g3-rs3-rs3-g3-gg5-g5-rs6-g6-e3']['car-id'].values.tolist()
    culp_dict = {}
    culp_dict[culp_ids[0]] = df_full_path[df_full_path['car-id'] == culp_ids[0]]['car-path'].iloc[0].split("-")
    fig = DrawGraph(culp_dict)
    fig.update_layout(width = 800, height = 500)
    fig.update_layout(autosize=True)
    return fig

def four_axle_culp_graph_2():
    filtered_df = df_full_path
    culp_ids = filtered_df[filtered_df['car-path'] == 'e3-g6-rs6-g5-gg5-g3-rs3-rs3-g3-gg5-g5-rs6-g6-e3']['car-id'].values.tolist()
    culp_dict = {}
    culp_dict[culp_ids[1]] = df_full_path[df_full_path['car-id'] == culp_ids[1]]['car-path'].iloc[0].split("-")
    fig = DrawGraph(culp_dict)
    fig.update_layout(width = 800, height = 500)
    fig.update_layout(autosize=True)
    return fig


QUESTIONABLE ACTIVITY LAYOUT AND GRAPH CREATION

In [200]:
def create_q_activity_layout():
    return html.Div([
        Header(),
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    # This is the card for the first bar chart
                    dbc.CardBody([
                        html.H3("Top Squatters", style={"text-align": "center"}),
                        html.Br([]),
                        dcc.Graph(
                            figure = top_squatters(),
                        )
                    ]), style={
                        "height": "700px","margin-left": "12px",
                               "margin-right": "12px",
                               "margin-top": "24px",
                               }
                )],
                width=4),
            dbc.Col([
                dbc.Card(
                    # This is the card for the second pie chart
                    dbc.CardBody([
                        html.H3("Timeline of Longest Squatter", style={"text-align": "center"}),
                        html.Br([]),
                        html.Div(
                            dcc.Graph(
                                figure = timeline_main_squatter(),
                            ), style={'overflowY':'scroll', 'height':500}
                        )
                    ]), style={"height": "700px","margin-left": "12px",
                               "margin-right": "24px",
                               "margin-top": "24px",
                               }
                )],
                width=8),
        ]),
        dbc.Row([
            dbc.Col([
                dbc.Card(
                    # This is the card for the first bar chart
                    dbc.CardBody([
                        html.H3("Illegal Edge Traversal/Possible Offroading", style={"text-align": "center"}),
                        html.Br([]),
                        dcc.Graph(
                            figure = illegal_edge_traversal(),
                        )
                    ]), style={"height": "600px","margin-left": "24px",
                               "margin-right": "12px",
                               "margin-top": "24px",
                               }
                )],
                width=6),
            dbc.Col([
                dbc.Card(
                    # This is the card for the first bar chart
                    dbc.CardBody([
                        html.H3("Joy Riding", style={"text-align": "center"}),
                        html.Br([]),
                        dcc.Graph(
                            figure = joy_riding(),
                        )
                    ]), style={"height": "600px","margin-left": "24px",
                               "margin-right": "12px",
                               "margin-top": "24px",
                               }
                )],
                width=6)
        ]),
        
    ])


# =============================================================================
# UPDATE QACTIVITY GRAPHS SECTION
# =============================================================================


def joy_riding():
    filtered_df = df_full_path
    filtered_df = filtered_df[(filtered_df['car-type'] != '2P')]
    filtered_df = filtered_df[filtered_df['car-path'].str.contains('c') == False]
    filtered_df = filtered_df.sort_values(by = 'duration-stay-hours', ascending = False).head(12)
    fig = px.bar(filtered_df, x= "duration-stay-hours", y = "car-id", orientation = 'h', color = "car-type", hover_data = ["start-ts", "end-ts", "car-path"])
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig


# Bar chart
# No call back required - UNUSUAL ACTIVITY
def top_squatters():
    filtered_df = df_full_path[["start-ts", "end-ts",'car-id','car-type','car-path', 'duration-stay']]
    filtered_df = filtered_df.sort_values(by = 'duration-stay', ascending = False).head(10).sort_values(by = 'duration-stay', ascending = True)
    filtered_df['start-ts'] = filtered_df['start-ts'].dt.strftime('%d/%m/%Y')
    filtered_df['end-ts'] = filtered_df['end-ts'].dt.strftime('%d/%m/%Y')
    filtered_df.columns = ['First seen in Park', 'Last-Seen in Park', 'Car-ID', 'Car-Type', 'Car-Path', 'Duration Stay (in days)']
    filtered_df['Duration Stay (in days)'] = np.round(filtered_df['Duration Stay (in days)'],0)
    fig = px.bar(filtered_df, x= 'Duration Stay (in days)', y = 'Car-ID', orientation = 'h', hover_data = ['First seen in Park', 'Last-Seen in Park'])
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig


# Illegal Edge Traversal
def illegal_edge_traversal():
    illegal_traverse_ids = df_speeding[(df_speeding['gate-name'] == 'entrance1') & (df_speeding['next_gate'] == 'ranger-stop1')]['car-id'].tolist()
    df_illegal = df_speeding[df_speeding['car-id'].isin(illegal_traverse_ids)]
    df_illegal['Timestamp'] = pd.to_datetime(df_illegal['Timestamp'])
    df_illegal['next_timestamp'] = pd.to_datetime(df_illegal['next_timestamp'])
    df_illegal['Date'] = df_illegal['Timestamp'].dt.strftime('%m/%d/%Y')
    fig = px.bar(df_illegal, y = "car-id", x = "path",orientation='h', color = "path", hover_data = ['Date'])
    fig.update_layout(xaxis_nticks = 20, yaxis_nticks = 40, height = 400)
    fig.update_layout(autosize=True)
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig

def timeline_main_squatter():
    filtered_df = df_speeding[df_speeding['car-id'] == '20155705025759-63']
    filtered_df = filtered_df[['Timestamp', 'next_timestamp', 'gate-name']]
    filtered_df['Timestamp'] = pd.to_datetime(filtered_df['Timestamp'])
    filtered_df['next_timestamp'] = pd.to_datetime(filtered_df['next_timestamp'])
    fig = px.timeline(filtered_df, x_start = 'Timestamp', x_end = 'next_timestamp',y = 'gate-name', color = 'gate-name')
    fig.update_layout(
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=True),
        yaxis=dict(zeroline=False, gridcolor='white'),
        paper_bgcolor='rgb(233,233,233)',
        plot_bgcolor='rgb(233,233,233)',
    )
    return fig



RUN APP FROM HERE

In [201]:
app.layout = html.Div([
                dcc.Location(id='url', refresh=True),
                html.Div(id='page-content')
])
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])

def display_page(pathname):
    if pathname == '/':
        return create_eda_layout()
    elif pathname == '/speeding':
        return create_speeding_layout()
    elif pathname == '/camping':
        return create_camping_layout()
    elif pathname == '/rangers':
        return create_ranger_layout()
    elif pathname == '/night_visits':
        return create_n_visits_layout()
    elif pathname == '/q-activity':
        return create_q_activity_layout()
    else:
        return '404'
    
app.run_server(mode = 'external', port=7090)

Dash app running on http://127.0.0.1:7090/
